In [4]:

!pip3 install pyspark

Found existing installation: pyspark 4.0.1
Can't uninstall 'pyspark'. No files were found to uninstall.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("jfktosea").getOrCreate() # name should be quniue for less conflicts

ModuleNotFoundError: No module named 'py4j'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, min, col, round, isnan

# --- 1. Initialize Spark Session ---
# Create a SparkSession instance.
spark = SparkSession.builder \
    .appName("JFK_to_SEA_Airtime_Analysis_SQL") \
    .getOrCreate()

# Set logging level to WARN to reduce console output clutter
spark.sparkContext.setLogLevel("WARN")

# --- 2. Define Schema and Load Data ---
# IMPORTANT: Replace 'path/to/your/nyc_flights.csv' with the actual file path.
FILE_PATH = 'path/to/your/nyc_flights.csv'

try:
    # Load the flights data
    flights_df = spark.read.csv(
        FILE_PATH,
        header=True,
        inferSchema=True
    )
    print(f"Successfully loaded data from: {FILE_PATH}")
    
    # --- 3. Register DataFrame as a Temporary SQL View ---
    # We must register the DataFrame so that Spark SQL can query it.
    flights_df.createOrReplaceTempView("flights")
    print("DataFrame registered as SQL view 'flights'.")

    # --- 4. Define and Execute Spark SQL Query (Challenge 3) ---
    # Challenge 3: Find the average, maximum, and minimum airtime for the JFK to SEA route.
    
    # The query filters by Origin='JFK' and Dest='SEA'.
    # It casts 'air_time' to INT and filters out NULLs to ensure accurate aggregation.
    query3 = """
    SELECT
        ROUND(AVG(CAST(air_time AS INT)), 2) AS Average_AirTime_Minutes,
        MAX(CAST(air_time AS INT)) AS Maximum_AirTime_Minutes,
        MIN(CAST(air_time AS INT)) AS Minimum_AirTime_Minutes
    FROM
        flights
    WHERE
        origin = 'JFK' AND dest = 'SEA' AND air_time IS NOT NULL
    """

    print("\n--- Executing Spark SQL Query for JFK -> SEA Airtime Statistics ---")
    result3 = spark.sql(query3)

    # --- 5. Display Results ---
    # The result will be a single row containing the three required metrics.
    result3.show(truncate=False)

    # Optional: Count total valid flights (using the registered view for count)
    count_query = """
    SELECT COUNT(*) AS total_flights
    FROM flights
    WHERE origin = 'JFK' AND dest = 'SEA' AND air_time IS NOT NULL
    """
    count_df = spark.sql(count_query)
    total_flights = count_df.collect()[0]['total_flights']
    print(f"\nTotal valid JFK -> SEA flights included in this analysis: {total_flights}")


except Exception as e:
    print(f"\n--- ERROR: Could not process the file. ---")
    print(f"Please check the FILE_PATH: '{FILE_PATH}' and ensure your Spark environment is set up correctly.")
    print(f"Detailed Error: {e}")

# --- 6. Stop Spark Session ---
spark.stop()

ModuleNotFoundError: No module named 'py4j'